### All the required imports

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import razor.flow as rf
from razor.flow import Pipeline
from razor.api import project_space_path
import typing as t

### Downloading and saving Fashion MNIST dataset into Project Space

### Load Images Block

In [8]:
@rf.block(executor=rf.ContainerExecutor(cores=1, memory=10000))
class LoadImages:
    path: str
    no_of_examples: int
    data: rf.SeriesOutput[dict]
    
    from razor.api import project_space_path
    def run(self):
        self.logger.debug('Running Load Images')
        
        npy = np.load(project_space_path(self.path))
            
        for image, label in zip(npy['images'][:self.no_of_examples], npy['labels'][:self.no_of_examples]):
            image = image.astype('float32') / 255.0
            label = np.eye(10)[[label]].reshape(-1)
            image = np.reshape(image, (image.shape[0], image.shape[1], 1))
            
            self.data.put({"images":image, "labels":label})
            
        self.logger.debug('Completed Load Images')

In [1]:
##aios60

@rf.block
class LoadImages:
    path: str
    no_of_examples: int
    
    def run(self):
        self.logger.debug('Running Load Images')
        
        npy = np.load(self.path)
        
        def generator():
            for image, label in zip(npy['images'][:self.no_of_examples], npy['labels'][:self.no_of_examples]):
                image = image.astype('float32') / 255.0
                label = np.eye(10)[[label]].reshape(-1)
                image = np.reshape(image, (image.shape[0], image.shape[1], 1))

                yield {"images":image, "labels":label}
                
        
            
        self.logger.debug('Completed Load Images')

NameError: name 'rf' is not defined

### Model

In [9]:
from razor.api import dlmodels
model = dlmodels.import_model(model_name='model_a', display=True)

INFO:rztdl.dl.model:Input Layer 'images'  validated successfully
INFO:rztdl.dl.model:BatchNormalization Layer 'BatchNormalization__1' validated successfully
INFO:rztdl.dl.model:Conv2D Layer 'Conv2D__1' validated successfully
INFO:rztdl.dl.model:MaxPool2D Layer 'MaxPool2D__1' validated successfully
INFO:rztdl.dl.model:Dropout Layer 'Dropout__1' validated successfully
INFO:rztdl.dl.model:Conv2D Layer 'Conv2D__2' validated successfully
INFO:rztdl.dl.model:MaxPool2D Layer 'MaxPool2D__2' validated successfully
INFO:rztdl.dl.model:Dropout Layer 'Dropout__2' validated successfully
INFO:rztdl.dl.model:Flatten Operator 'Flatten__1' validated successfully
INFO:rztdl.dl.model:Dense Layer 'Dense__1' validated successfully
INFO:rztdl.dl.model:Dropout Layer 'Dropout__3' validated successfully
INFO:rztdl.dl.model:Dense Layer 'Dense__2' validated successfully
INFO:rztdl.dl.model:BatchNormalization Layer 'BatchNormalization__2' validated successfully
INFO:rztdl.dl.model:Dense Layer 'Dense__3' validated

import rztdl.dl
from rztdl.dl.constants.string_constants import *
from rztdl.dl.helpers import *
from rztdl.dl.helpers.dtypes import DataTypes
from rztdl.dl.groups import Group
from rztdl.dl.components.layers import *
from rztdl.dl.hub import *
from rztdl.dl.components.metrics import *
from rztdl.dl.components.operators import *
from rztdl.dl.components.losses import *
from rztdl.dl.components.layers.recurrent.cells import *
from rztdl.dl.components.optimizers import *

# CustomCodeTemplates



model = rztdl.dl.RZTModel(name = 'model_a')
model.add(Input(dtype=DataTypes.FLOAT32,validate_shape=True,shape=[28, 28, 1],name='images',outputs='images_outputs'))
model.add(BatchNormalization(inputs='images_outputs',axis=[-1],momentum=0.99,epsilon=0.001,center=False,scale=False,moving_variance_initializer=Ones(),trainable=True,name='BatchNormalization__1',outputs='BatchNormalization__1_outputs'))
model.add(Conv2D(inputs='BatchNormalization__1_outputs',kernel_size=[4, 4],strides=[1, 1],padding=Pa

#### Train Pipeline

In [10]:
loader = LoadImages(
              path='data_train.npz',
              no_of_examples=50000)

In [11]:
from razor.marketplace.blocks.rzt.train_flow import DLTrainBlock
trainer = DLTrainBlock(
               model=model,
               train_data=loader.data,
               epochs=50,
               optimizers=["adam1"],
               metrics=['cce','acc'],
               batch_size=512,
               save_path="image_classification_s3",
               log_frequency=20)
trainer.executor = rf.ContainerExecutor(cores=2, memory=20000)
trainer.saved_model = trainer.saved_model.set(transport=rf.KafkaTransport(is_series=False))

In [6]:
train_pipeline = rf.Pipeline(name = 'pipe_50k' ,targets=[trainer])

In [22]:
import razor
deployed_pipeline= razor.api.engines("DS-Engine-Load-Test").execute(train_pipeline)